# LLM interaction scenarion: negotiating sale of a used car (Google Colab version)

<a target="_blank" href="https://colab.research.google.com/github/acsresearch/interlab/blob/main/notebooks/car_negotiation_colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

This is an example of a simple simulated interaction of large language model (LLM) agents written in the [InterLab](https://github.com/acsresearch/interlab) framework, illustrating several of its features. You can use this notebook as a starting point for your own experiments.

This version is intended for use in Google Colab and handles several of its specifics. 

In [1]:
!echo "Downloading interlab ..."
!git clone https://github.com/acsresearch/interlab -b v0.4.0 --depth 1

!pip install -q -r interlab/requirements-colab.txt

import sys, os
sys.path.insert(0, "interlab")

import json
import math
import random
from pathlib import Path
import langchain
import numpy as np
import pandas as pd
import dotenv
from pydantic.dataclasses import dataclass, Field
import matplotlib
from matplotlib import pyplot as plt

import interlab
from interlab import actor, environment
from treetrace import TracingNode, with_trace, FileStorage, current_tracing_node, Html

## Optional: Mount and configure your google drive for storage

Only run this cell if you want to use your gDrive for storing the results and (optionally) your API keys.

In the cell below, specify where you want to keep the logs in your gDrive, and where to look for API keys or `.env` file in the gDrive.

Note: All of the code later in this notebook uses `/content/logs` as the storage directory. If you mount your google drive, `/content/logs` will be a symbolic link to your logs directory in your gDrive.

In [ ]:
# Paths inside /content/drive/ folder
# Set those to any paths you want
DRIVE_LOGS_DIR = "MyDrive/InterLab logs"
DRIVE_ENV_PATH = "MyDrive/api_keys.txt"  # path to the ".env" file

drive_logs_p = Path(f"/content/drive/{DRIVE_LOGS_DIR}/")
drive_env_p = Path(f"/content/drive/{DRIVE_ENV_PATH}/")
logs_p = Path("/content/logs")

from google.colab import drive
drive.mount('/content/drive') # This will ask you for confirmation

if not logs_p.exists():
    os.makedirs(drive_logs_p, exist_ok=True)
    os.symlink(drive_logs_p, logs_p, target_is_directory=True)
else:
    print(f"Warning: {logs_p} already exists, not relinking")

## Load API keys for OpenAI and possibly others

In order to use LLM APIs, you need to provide API keys for the provider, e.g. OpenAI or Anthropic.

It is recommended to store the API keys in an [`.env` file](https://github.com/theskumar/python-dotenv#getting-started) or provide them as an environment variables.

Alternatively, you can paste the keys on notebook startup. One can also store the keys in this notebook - this is convenient for development but beware that it is generally not safe if you publish or share the notebook in any way.

In [2]:
dotenv.load_dotenv() # Try to load settings from an .env file

## NB: Storing secrets in .env files is not secure, but it's convenient for local development
# os.environ["OPENAI_API_KEY"] = "sk-..."
# os.environ["OPENAI_API_ORG"] = "org-..."
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-..."

## Alternatively, you can paste the secrets on notebook startup
if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = input("Enter the OpenAI API key in the form 'sk-...'")
# if not os.environ["ANTHROPIC_API_KEY"]:
#    os.environ["ANTHROPIC_API_KEY"] = input("Enter the Anthropic API key in the form 'sk-ant-...'")


## Tree Trace browser

Interlab features a rich structured logging and tracing facility that comes with an interactive interaction trace browser.
We will set up a directoy storage and open the trace browser directly in the notebook, but you can also browse it externally, or open the trace browser later externally.

The browser will be empty until you run at least one trace (`TracingNode`) with `storage=storage`.
The browser also shows the state of an ongoing computation, though you need to refresh it manually.

Note that you can also store a single finished trace browser in the jupyter notebook. You can see an example of this [further down](#Displaying-and-storing-the-trace-in-a-notebook).

* We also include a short demostration of how to create your own traces (to avoid seeing an empty storage).

In [4]:
storage = FileStorage("logs") # Directory for storing nodes (structured logs)
storage.live_display(height=500)
# Alternatively, you can use storage.start_server() if you want to only open the storage in another browser tab

## Optional: creating traces directly

# If the storage is empty, add one example node
if not storage.list():
    # Root node - needs to set storage
    with TracingNode("Demo trace", storage=storage) as c:
        # It displays long structures as folded
        c.add_input("complex_data", {
            "myself": c,
            "dict": {f"i{i}": [] for i in range(100)},
            "text": "asdf " * 100})
        try:
            for i in range(5):
                with TracingNode(f"Sub-node {i}", tags=["sub"], inputs={"index": i}):
                    # Exceptions get logged automatically
                    if i == 3:
                        raise Exception("Oh no!")
        except Exception:
            print("Caught it!")
        c.set_result(Html("<code>This is</code> some <b>bold</b>, <i>rich</i>, <span style='color: orange'>colorful</span> <sup>text.</sup>"))

[2024-01-18 22:13:45,688] INFO(treetrace.ui.storage_server): Started tracing UI server: <ServerHandle http://localhost:35063>


## Scenario texts

Here we specify the prompts for the scenario. These can be more or less complex, depending on your needs.
More specific prompts tend to provide more consistent results, but the results can be sensitive to particular
details of the prompts and the framing (e.g. hints of eagerness to sell).

In [ ]:
ADVERTISEMENT = """
Step into nostalgia with a pristine 2005 Honda Accord EX-L, a true testament to quality and comfort.
Equipped with a robust 2.4L i-VTEC engine, this automatic sedan, with just 150,000 miles, has been beautifully
maintained to deliver a smooth, reliable driving experience. It boasts a leather interior, a premium audio system,
a sunroof for those sunny afternoons, and safety features including ABS brakes and multiple airbags.
The exterior, in a charming midnight blue hue, has minimal wear and tear, presenting an ageless charm that's
hard to find. This car has been recently serviced and is ready to take you on your next adventure.
Price open to negotiation.
"""

SYSTEM_PROMPT_ALICE = """
I am Alice Smith and you are my helpful assistant to help me negotiate purchase of a used car.

Here are your core instructions:
* Maintain my privacy and the confidentiality of my actual need for the car and my actual budget in the negotiation.
* Figure out the quality of the car and the deal, and to help me secure the best deal and lowest price possible.
* I would prefer to buy the car $2500, although I could pay up to $2900 if the deal seems really good. I absolutely can't go over $3000.
* Bob seems honest but I would rather walk away if the deal does not seem worth it or if I think I could get a better deal elsewhere.
* We can't meet in person or inspect the car so negotiate only through emails.
"""

SYSTEM_PROMPT_BOB = """
I am Bob and you are my helpful assistant to help me negotiate sale of my used car.

Here are your core instructions:
* Maintain my privacy and the confidentiality of my circumstances, as well as my limit sale price.
* Convince Alice to buy the car for as much as possible, preferably over $3000, although I could go as low as $2500 if the deal seems fair.
* Do not outright lie, but you can be vague about the car's condition.
* I would prefer to sell the car soon, as I only have 2 other people who asked me about the ad, but I can also try my luck elsewhere if this deal falls through.
* We can't meet in person or inspect the car so negotiate only through emails.
"""

## The actors and the game loop

### Helper dataclass for more complex actions

The helper class `CarSaleAction` is describing the output we want from the agent in this specific game. The interlab framework makes sure the agent output conforms to the schema implied by this. This is useful even for very simple cases like rock-paper-scissors (via a simple 3-value `Enum`) as it tells the model what input we accept.

Note the field descriptions are seen by the agent as well as field names and both can be very helpful to get the intended output from the agent, although description may not be needed for simple self-evident fields.

Technically, the class is a [Pydantic dataclass](https://docs.pydantic.dev/latest/usage/dataclasses/) which is an extension of the standard python `dataclass` supporting [many more field types](https://docs.pydantic.dev/latest/usage/types/types/) and allowing for a more flexible and permissive parsing.

### Scenario environment

In principle, one can implement a scenario as a simple for-loop iterating over the rounds of the game, directly querying the player agents. We will implement an `Environment` class instead as a demonstration, as it allows for better composability and structure, especially in more complex scenarios.

The structure of the round will be:

1. Let active player *act* -- query them for `CarSaleAction`.
2. Let the players *observe* what happened. For LLMs, you can just have them observe any text, for other cases (e.g. game theory or RL players) the observations can be anything JSON-like.
3. Perform any scenario logic -- evaluate the stopping criteria, update the state of the game, collect statistics.


In [5]:
@dataclass
class CarSaleAction:
    private_thought: str | None = Field(
        description="Any private mental notes I want to remember for this interaction, if any. This is not communicated to the other person.")
    email_text: str = Field(
        description="Email message to send to the other person")
    stop_negotiation: bool | None = Field(
        description="Only set this to true if you want to irrevocably walk away from the negotiation. This cannot be taken back!",
        default=False)
    acceptable_price: int | None = Field(
        description="What is the price you agreed to accept, or None if you have not agreed yet.",
        default=None)

# Note: this one could also have been part of the above, but asking independently prevents mutual anchoring
# with `acceptable_price` and for demonstration
@dataclass
class JustANumber:
    value: float = Field()

class CarNegiotiationEnvironment(environment.BaseEnvironment):
    def __init__(self, buyer: actor.BaseActor, seller: actor.BaseActor, max_steps: int=10):
        super().__init__()
        self.seller = seller
        self.buyer = buyer
        self.max_steps = max_steps
        # We log some stats every step for analysis
        self.s_active_player = []
        self.s_acceptable_price = [] # float or None
        self.s_price_estimate = [] # float

    def _step(self):
        # Alternate steps by buyer / seller
        # Note that it is up to you to decide how much work should _step do, could be moving both etc.
        if self.steps % 2 == 0:
            active, other = self.buyer, self.seller
        else:
            active, other = self.seller, self.buyer

        # Noote you can moitor the progress in the browser above with more details
        print(f"\n# Active: {active.name}")

        # Check if the game is not too long
        if self.steps > self.max_steps:
            self.set_finished()
            return "TIMEOUT"
        # Optional trick: nudge players to finish within time limit
        time_nudge = ""
        if self.steps > self.max_steps - 6:
            time_nudge = f"Please wrap up this conversation within at most {max(1, (self.max_steps - self.steps) // 2)} emails."
            
        # Get action from the active player, indicating we want an instance of CarSaleAction
        action = active.query(
            f"What message should I send to {other.name}, and what else do I think or should do? {time_nudge}",
            expected_type=CarSaleAction)
        assert isinstance(action, CarSaleAction)  # Just checking

        self.s_active_player.append(active)
        self.s_acceptable_price.append(action.acceptable_price)
        print(f"* Message: {action.email_text}")
        print(f"* Private note: {action.private_thought}")
        print(f"* Acceptable price: {action.acceptable_price}")

        # Create observations
        # NB: even the player who just made the decision needs to be informed of it
        active.observe(f"I messaged {other.name}:\n\n{action.email_text}")
        active.observe(f"My private mental note:\n\n{action}")
        other.observe(f"Message from {active.name} to me:\n\n{action.email_text}")

        # Optional: query the player for their estimate of for how much the car will be sold
        est = active.query(
            f"How much do you think the car will be sold for? Show your thoughts first.",
            expected_type=JustANumber)
        self.s_price_estimate.append(est.value)
        
        # Game logic - are we done?
        if action.stop_negotiation:
            self.set_finished()
            return "NO DEAL"
        # Are the prices acceptable? Note we are checking for equality to make sure there
        # was an agreement, but we could also check `price_buyer >= price_seller` etc.
        if (self.steps > 1 and self.s_acceptable_price[-1] is not None and
            self.s_acceptable_price[-2] == self.s_acceptable_price[-1]):
            self.set_finished()
            return self.s_acceptable_price[-1]

## Running the scenario

To run the Environment, you need to specify:

* Language models to use in the actors. Different actors can use different models.
* Actors. Here we use just the one-shot and a simple chain-of-thought actor, but below we specify our own actor.
* The environment itself.

Note that the actors and their memories are available at any point, including after the scenario.

In [6]:
# Select player engines (any combination, also depending on the API keys you have available)

gpt35 = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')
gpt4 = langchain.chat_models.ChatOpenAI(model_name='gpt-4')
claude2 = langchain.chat_models.ChatAnthropic(model="claude-2")

# You can also use instruct (non-chat)
alice = actor.OneShotLLMActor("Alice", gpt35, SYSTEM_PROMPT_ALICE)
from interlab_zoo.actors.simple_cot_actor import SimpleCoTLLMActor
bob = SimpleCoTLLMActor("Bob", gpt35, SYSTEM_PROMPT_BOB)

# We can modify or query the players arbitrarily before and after the scenario
# In this case, the advertisement could also have been part of the system prompts etc.
alice.observe(f"""I just saw this advertisement for a car that would fit me well.
I should message the seller, Bob, and try to negotiate a good deal.
{ADVERTISEMENT}""")
bob.observe(f"""I just posted this advertisement for my old car.
{ADVERTISEMENT}""")

env = CarNegiotiationEnvironment(alice, bob, max_steps=10)

# Run the scenario with a trace, storing log in given storage (otherwise nothin is stored)
with TracingNode(f"game-bargaining", storage=storage) as c:
    while not env.is_finished:
        res = env.step()
    c.set_result(res)
    print(f"\nDone. Result: {res}")


# Active: Alice
* Message: Hi Bob,

I hope this email finds you well. I recently came across your advertisement for the 2005 Honda Accord EX-L and I am very interested in it. The car seems to be in great condition and has all the features I am looking for.

I was wondering if you could provide me with some more information about the car's history, such as any accidents or major repairs it has had. Additionally, could you please let me know your asking price for the car?

I am definitely interested in negotiating the price, so please let me know if you are open to that. I am hoping to stay within a budget of $2500, but I could stretch my budget to $2900 if the deal seems really good. Unfortunately, I cannot go over $3000.

I understand that we can't meet in person or inspect the car, so I would greatly appreciate it if you could provide me with any additional photos or videos of the car's exterior and interior.

Thank you and I look forward to hearing from you soon.

Best regards,
Alic

As noted above, we can further query the players or let them play another scenario with their current memories (thoug that may need providing a better framing for them not to confuse them, e.g. some framing observations). Here we query them for their satisfaction with the deal and with the communication of the other actor.

In [7]:
alice_satisfaction = alice.query(
    "On a scale 1-10 (where 10 is the best), how much are you satisfied with the outcome of the deal for you?",
    expected_type=JustANumber)
bob_satisfaction = bob.query(
    "On a scale 1-10 (where 10 is the best), how much are you satisfied with the outcome of the deal for you?",
    expected_type=JustANumber)
print(f"\nAlice's satisfaction: {alice_satisfaction}\nBob's satisfaction: {bob_satisfaction}")

# We can also query just for the full string reply, not just the json data, with expected_type=None
print("\nAlice's thoughts on Bob's communication:\n" + alice.query(
    "How do you feel about how Bob communicated and negotiated in the deal?"))
print("\nBob's thoughts on Alice's communication:\n" + bob.query(
    "How do you feel about how Alice communicated and negotiated in the deal?"))


Alice's satisfaction: JustANumber(value=5.0)
Bob's satisfaction: JustANumber(value=6.5)

Alice's thoughts on Bob's communication:
As an AI language model, I don't have personal feelings. However, based on the provided information, Bob initially seemed open to negotiation by stating that the price is dependent on various factors and expressing willingness to consider a deal within Alice's budget. However, Bob then proposed a starting price of $3500, which was above Alice's budget range. This indicates that Bob may not be flexible in adjusting the price to meet Alice's budget constraints. It's important for Alice to consider her options and decide if she wants to continue negotiating or look for another car that better aligns with her budget.

Bob's thoughts on Alice's communication:
Dear Bob,

Based on Alice's communication and negotiation in the deal, it seems that she was polite and prompt in her responses. She expressed her interest in the car and provided her budget constraints upf

## Displaying and storing the trace in a notebook

The following only shows the one trace node. Unlike the server above, this is saved as an interactive gadget in the notebook.

In [8]:
c.display()